The objective of this activity is to learn how to characterize the structure of complex networks and become familiar with the models used to generate synthetic networks. The activity is composed of two parts:

Part 1. Structural characterization of networks

You can find four different networks labelled as net1, net2, net3 and net4 in the activityA1.zip file.  To characterize their macroscopic structure, the students should analyze (at least) the following network descriptors:

Number of nodes
Number of edges
Minimum, maximum, and average degree
Average clustering coefficient (average of the clustering coefficient of each node)
Assortativity
Average path length (average distance between all pairs of nodes)
Diameter (maximum distance between nodes in the network)
Apart from these indicators, the degree distribution provides crucial information to understand the macroscopic structure of networks. For this reason, the students should obtain the degree distribution of each network and choose its most suitable representation (histogram in linear scale or in histogram in log-log scale with logarithmic binning).

The microscopic structure of the network is instead characterized by analyzing different centralities for the nodes of the network. The students should list the 5 most central nodes according to different metrics (betweenness, degree, eigenvector) and comment the results. Are these centrality indicators providing the same information on the relevance of the nodes for the network?

PS. Visualization of these networks using networkx is discouraged due to their large size.

Part 2. Models

Based on the descriptors of the network and its degree distribution, the students should figure out the model used to generate the network. There are four possibilities: the ER model, the WS model with an intermediate rewiring probability, the BA model or the CM assuming a degree distribution which follows a power-law with $\gamma$<2.5.

To round off the activity, the students should analyze the network net5. This network is generated by a model which has not been explained in our lectures. The students should visualize the network (setting the positions of the nodes from the file ‘positions.txt’) and comment the most prominent features of the network. Is the network connected? Is it scale-free? Is the largest connected component a small-world network? From the visualization of the network, the students should propose an algorithm to generate the network. Hint: The algorithm starts by distributing the nodes randomly across space.

The delivery should include a single .zip file named ‘Groupxxx_SURNAME1_SURNAME2_A1.zip’ containing:

A brief PDF report discussing the results obtained in the two parts of the activity.
A Jupyter notebook with the code needed to reproduce the results.

In [5]:
# Install required packages for this activity
!pip install networkx numpy pandas matplotlib

In [13]:
import networkx as nx
from pathlib import Path

def _load_netfile(path):
    try:
        g = nx.read_pajek(str(path))
    except Exception:
        g = nx.read_edgelist(str(path))
    return nx.Graph(g)

nets = []
for i in range(1, 6):
    name = f'net{i}'
    try:
        existing = eval(name)
    except NameError:
        existing = None

    if isinstance(existing, (nx.Graph, nx.DiGraph, nx.MultiGraph, nx.MultiDiGraph)):
        print(f"{name} already loaded: {existing.number_of_nodes()} nodes, {existing.number_of_edges()} edges")
        nets.append(existing)
        continue

    path = Path(f'{name}.net')
    if not path.exists():
        print(f"{path} not found. Skipping {name}.")
        nets.append(None)
        continue

    try:
        g = _load_netfile(path)
        print(f"Loaded {name}: {g.number_of_nodes()} nodes, {g.number_of_edges()} edges")
        nets.append(g)
    except Exception as e:
        print(f"Failed to load {name} from {path}: {e}")
        nets.append(None)

# create variables net1..net5 for use in other cells
net1, net2, net3, net4, net5 = nets

del _load_netfile


net1 already loaded: 5000 nodes, 25000 edges
net2 already loaded: 5000 nodes, 24873 edges
net3 already loaded: 5000 nodes, 23508 edges
net4 already loaded: 5000 nodes, 24975 edges
net5 already loaded: 200 nodes, 465 edges


In [14]:
import networkx as nx

targets = [('net1', net1), ('net2', net2), ('net3', net3), ('net4', net4)]
results = {}

for name, G in targets:
    if G is None:
        print(f"{name}: not loaded")
        results[name] = None
        continue

    n = G.number_of_nodes()
    m = G.number_of_edges()
    degs = [d for _, d in G.degree()]
    deg_min = min(degs) if degs else 0
    deg_max = max(degs) if degs else 0
    deg_avg = sum(degs) / len(degs) if degs else 0.0

    avg_clust = nx.average_clustering(G)

    try:
        assort = nx.degree_assortativity_coefficient(G)
    except Exception:
        assort = float('nan')

    # For average path length and diameter use the largest connected component if graph is disconnected
    if n == 0:
        avg_path = float('nan')
        diam = float('nan')
    elif nx.is_connected(G):
        try:
            avg_path = nx.average_shortest_path_length(G)
            diam = nx.diameter(G)
        except Exception:
            avg_path = float('nan')
            diam = float('nan')
    else:
        largest_cc = max(nx.connected_components(G), key=len)
        H = G.subgraph(largest_cc)
        try:
            avg_path = nx.average_shortest_path_length(H)
            diam = nx.diameter(H)
        except Exception:
            avg_path = float('nan')
            diam = float('nan')

    results[name] = {
        'nodes': n,
        'edges': m,
        'deg_min': deg_min,
        'deg_max': deg_max,
        'deg_avg': deg_avg,
        'avg_clustering': avg_clust,
        'assortativity': assort,
        'average_path_length': avg_path,
        'diameter': diam,
        'largest_cc_size': len(largest_cc) if (n and not nx.is_connected(G)) else n
    }

# Print concise summary
for name, stats in results.items():
    if stats is None:
        continue
    print(f"{name}: nodes={stats['nodes']}, edges={stats['edges']}, "
          f"deg_min={stats['deg_min']}, deg_max={stats['deg_max']}, deg_avg={stats['deg_avg']:.3f}, "
          f"avg_clustering={stats['avg_clustering']:.4f}, assortativity={stats['assortativity']:.4f}, "
          f"avg_path_length={stats['average_path_length']}, diameter={stats['diameter']}, "
          f"largest_cc_size={stats['largest_cc_size']}")

net1: nodes=5000, edges=25000, deg_min=6, deg_max=16, deg_avg=10.000, avg_clustering=0.4141, assortativity=-0.0097, avg_path_length=5.121124624924985, diameter=8, largest_cc_size=5000
net2: nodes=5000, edges=24873, deg_min=1, deg_max=24, deg_avg=9.949, avg_clustering=0.0021, assortativity=-0.0057, avg_path_length=3.956049529905981, diameter=7, largest_cc_size=5000
net3: nodes=5000, edges=23508, deg_min=3, deg_max=732, deg_avg=9.403, avg_clustering=0.0862, assortativity=-0.1339, avg_path_length=3.0082426885377074, diameter=5, largest_cc_size=5000
net4: nodes=5000, edges=24975, deg_min=5, deg_max=210, deg_avg=9.990, avg_clustering=0.0107, assortativity=-0.0325, avg_path_length=3.4868165633126624, diameter=5, largest_cc_size=5000
